<a href="https://colab.research.google.com/github/saishshinde15/FineTuning/blob/main/Summmary_Model_Llama_3_2_1B_Instruct_finetuning(Guideee_%2C_Refreee_This.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "True",
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # LoftQ
)

Unsloth 2024.9.post3 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
summary_prompt = """
You are given the following text. Please provide a summary in 5-10 key points, depending on the length of the document. Each point should be clearly formatted in bullet format, starting with an asterisk (*).

**Note:** The examples provided below are for your reference only and should not be included in your response.

### Examples (for reference only):
* The sky is blue on a clear day.
* Water boils at 100 degrees Celsius.
* Trees produce oxygen through photosynthesis.

### Original Text:
{}

### Key Points Summary (in bullet points):
"""
EOS_TOKEN = tokenizer.eos_token  # Ensure end of sequence

def formatting_prompts_func(examples):
    texts    = examples["info"]     # 'info' column contains the text to be summarized
    summaries = examples["summaries"] # 'summary' column contains the summaries

    formatted_texts = []
    for text, summary in zip(texts, summaries):
        # Format the prompt with text and summary
        formatted_text = summary_prompt.format(text, summary) + EOS_TOKEN
        formatted_texts.append(formatted_text)

    return { "text": formatted_texts }

# Load the dataset (adjust as needed)
from datasets import load_dataset
dataset = load_dataset("openai/summarize_from_feedback", "comparisons",split="train")

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/92858 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/92858 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.781 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 92,858 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,3.693000
2,3.226100
3,3.332900
4,3.016400
5,2.815400
6,2.836300
7,2.519200
8,2.646900
9,2.438600
10,2.302700


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

184.4213 seconds used for training.
3.07 minutes used for training.
Peak reserved memory = 3.621 GB.
Peak reserved memory for training = 0.84 GB.
Peak reserved memory % of max memory = 24.552 %.
Peak reserved memory for training % of max memory = 5.696 %.


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    summary_prompt.format(
        "Document AI is a Snowflake AI feature that uses Arctic-TILT, a proprietary large language model (LLM), to extract data from documents. Document AI processes documents of various formats and extracts information from both text-heavy paragraphs and the content in a graphical form, such as logos, handwritten text (signatures), or checkmarks. With Document AI, you can prepare pipelines for continuous processing of new documents of a specific type, such as invoices or finance statements.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1280)

<|begin_of_text|>
You are given the following text. Please provide a summary in 5-10 key points, depending on the length of the document. Each point should be clearly formatted in bullet format, starting with an asterisk (*).

**Note:** The examples provided below are for your reference only and should not be included in your response.

### Examples (for reference only):
* The sky is blue on a clear day.
* Water boils at 100 degrees Celsius.
* Trees produce oxygen through photosynthesis.

### Original Text:
Document AI is a Snowflake AI feature that uses Arctic-TILT, a proprietary large language model (LLM), to extract data from documents. Document AI processes documents of various formats and extracts information from both text-heavy paragraphs and the content in a graphical form, such as logos, handwritten text (signatures), or checkmarks. With Document AI, you can prepare pipelines for continuous processing of new documents of a specific type, such as invoices or finance statements.

In [ ]:
summary_prompt = """
You are given the following text. Please provide a summary in 5-10 key points, depending on the length of the document. Each point should be clearly formatted in bullet format, starting with an asterisk (*).

**Note:** The examples provided below are for your reference only and should not be included in your response.

### Examples (for reference only):
* The sky is blue on a clear day.
* Water boils at 100 degrees Celsius.
* Trees produce oxygen through photosynthesis.

### Original Text:
{}

### Key Points Summary (in bullet points):
"""

# Enable 2x faster inference with the model
FastLanguageModel.for_inference(model)

# Prepare the inputs for the tokenizer by formatting the Alpaca prompt for summarization
inputs = tokenizer(
    [
        summary_prompt.format(
            "Document AI is a Snowflake AI feature that uses Arctic-TILT, a proprietary large language model (LLM), to extract data from documents. Document AI processes documents of various formats and extracts information from both text-heavy paragraphs and the content in a graphical form, such as logos, handwritten text (signatures), or checkmarks. With Document AI, you can prepare pipelines for continuous processing of new documents of a specific type, such as invoices or finance statements.",  # Text to summarize
        )
    ],
    return_tensors="pt"  # Convert input to PyTorch tensors
).to("cuda")  # Move input tensors to GPU for faster processing

# Initialize a text streamer to print generated text (summary) as it is created
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

# Generate the summary, streaming the output in real-time
_ = model.generate(
    **inputs,  # Input tensors for the model
    streamer=text_streamer,  # Stream output as it's generated
    max_new_tokens=128  # Limit the number of tokens for the summary to 128
)


<|begin_of_text|>
You are given the following text. Please provide a summary in 5-10 key points, depending on the length of the document. Each point should be clearly formatted in bullet format, starting with an asterisk (*). 

**Note:** The examples provided below are for your reference only and should not be included in your response.

### Examples (for reference only):
* The sky is blue on a clear day.
* Water boils at 100 degrees Celsius.
* Trees produce oxygen through photosynthesis.

### Original Text:
Document AI is a Snowflake AI feature that uses Arctic-TILT, a proprietary large language model (LLM), to extract data from documents. Document AI processes documents of various formats and extracts information from both text-heavy paragraphs and the content in a graphical form, such as logos, handwritten text (signatures), or checkmarks. With Document AI, you can prepare pipelines for continuous processing of new documents of a specific type, such as invoices or finance statements

In [ ]:
!huggingface-cli login

model.push_to_hub("saishshinde15/Summmary_Model_Llama-3.2-1B-Instruct", check_pr=True)

tokenizer.push_to_hub("saishshinde15/Summmary_Model_Llama-3.2-1B-Instruct",check_pr=True)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


README.md:   0%|          | 0.00/602 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Saved model to https://huggingface.co/saishshinde15/Summmary_Model_Llama-3.2-1B-Instruct


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model.save_pretrained("Summmary_Model_Llama-3.2-1B-Instruct") # Local saving
tokenizer.save_pretrained("Summmary_Model_Llama-3.2-1B-Instruct")
#model.push_to_hub("saishshinde15/Summmary_Model_Llama-3.2-1B-Instruct", token = "...") # Online saving
#tokenizer.push_to_hub("saishshinde15/Summmary_Model_Llama-3.2-1B-Instruct", token = "...") # Online saving

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


('Summmary_Model_Llama-3.2-1B-Instruct/tokenizer_config.json',
 'Summmary_Model_Llama-3.2-1B-Instruct/special_tokens_map.json',
 'Summmary_Model_Llama-3.2-1B-Instruct/tokenizer.json')

In [ ]:
from google.colab import files. # push these files manualy in hf
!zip -r /content/Summmary_Model_Llama-3.2-1B-Instruct.zip /content/Summmary_Model_Llama-3.2-1B-Instruct
files.download("/content/Summmary_Model_Llama-3.2-1B-Instruct.zip")


  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/ (stored 0%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/special_tokens_map.json (deflated 71%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/config.json (stored 0%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/generation_config.json (deflated 33%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/model.safetensors (deflated 8%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/tokenizer.json (deflated 85%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/tokenizer_config.json (deflated 94%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/adapter_config.json (deflated 54%)
  adding: content/Summmary_Model_Llama-3.2-1B-Instruct/README.md (deflated 48%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# config.json file (also push manually)
config = {
  "_name_or_path": "meta-llama/Llama-3.2-3B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": False,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "vocab_size": 32000
}

import json

with open("config.json", "w") as f:
  json.dump(config, f, indent=4)


In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the model and tokenizer
model_id = "saishshinde15/Summmary_Model_Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Define the summary prompt
summary_prompt = """
You are given the following text. Please provide a summary in 5-10 key points, depending on the length of the document. Each point should be clearly formatted in bullet format, starting with an asterisk (*).

**Note:** The examples provided below are for your reference only and should not be included in your response.

### Examples (for reference only):
* The sky is blue on a clear day.
* Water boils at 100 degrees Celsius.
* Trees produce oxygen through photosynthesis.

### Original Text:
{}

### Key Points Summary (in bullet points):
"""

def summarize(text):
    # Prepare the inputs for the tokenizer by formatting the summary prompt
    inputs = tokenizer(
        summary_prompt.format(text),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512  # Adjust max_length if needed
    ).to("cuda")

    # Generate the summary
    generated_summary = model.generate(
        **inputs,
        max_new_tokens=128,  # Limit the number of tokens for the summary
        do_sample=True,
    )

    # Decode the generated summary
    summary = tokenizer.decode(generated_summary[0], skip_special_tokens=True)
    return summary

# Create Gradio interface
iface = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(label="Enter Text to Summarize", lines=10),
    outputs=gr.Textbox(label="Summary"),
    title="Text Summarization",
    description="Enter any text, and the model will provide a summary in bullet points.",
)

# Launch the app
iface.launch()


config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://df8e8056619a4460e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
